# Visualization of Parameter, Compute and Data Trends in Machine Learning

This is an accompanying notebook to [this dataset](https://docs.google.com/spreadsheets/d/1AAIebjNsnJj_uKALHbXNfn3_YsT6sHXtCU0q7OIPuc4/edit#gid=0).

Please cite us as *Parameter, Compute and Data Trends in Machine Learning* 
by Jaime Sevilla, Pablo Villalobos, Juan Felipe Cerón, Matthew Burtell, Lennart Heim, Amogh B. Nanjajjar, Anson Ho, Tamay Besiroglu and Marius Hobbhahn.

In [ ]:
#@title Retrieve data and plot interactive graph
import pandas as pd
import numpy as np
from scipy import stats
import numpy as np
from sklearn import linear_model
import altair as alt


## PARAMETERS
x_axis = 'Publication date' #@param ['Publication date', 'Parameters', 'Training compute (FLOPs)', 'Inference compute (FLOPs)', 'Training compute per parameter (FLOPs)', 'Training compute times parameters']
y_axis = 'Training compute (FLOPs)' #@param ['Parameters', 'Training compute (FLOPs)', 'Inference compute (FLOPs)', 'Training compute per parameter (FLOPs)', 'Training compute times parameters', 'Training cost (2020 USD)']
date_start = '1950-01-01'# '2017-12-06' # '2012-09-29'  #'2009-12-30' #  2015-09-01' # #@param 
date_end = '2022-02-20' # '2017-12-31'# #@param 

#@markdown Era options
start_dl_era = '2009-12-31' # "2012-09-30"# #@param 
start_large_scale_era = '2012-09-30' # '2017-12-06' #'2015-09-01' # #@param
split_dl_era = True #@param {'type':'boolean'}
split_large_scale_era = True #@param {'type':'boolean'}

#@markdown Data options
citation_threshold = 0 #@param
separate_categories = False #@param {'type':'boolean'}
other_domain_threshold = 10
outliers_action = 'label' #@param ['ignore', 'label', 'remove']
large_scale_action = 'ignore' #@param ['ignore', 'label', 'isolate']
big_alphago_action = 'ignore' #@param ['ignore', 'label', 'remove']
record_setters_action = 'label' #@param ['ignore', 'label', 'isolate']

#@markdown Choose Z-value thresholds to automatically detect outliers
low_outliers_z_value_threshold = -2 #@param
high_outliers_z_value_threshold = 0.76 #@param
outlier_window_size = 2 #@param

#@markdown Bootstrapping options
bootstrap_sample_size = 1000 #@param
adjust_for_estimate_uncertainty = True #@param {'type':'boolean'}

#@markdown Visualization options 
label_points = False #@param {'type' : 'boolean'}
plot_regressions = True #@param {'type' : 'boolean'}
label_eras = True #@param {'type' : 'boolean'}

## CODE
def make_visualization(
    x_axis = 'Publication date',
    y_axis = 'Training compute (FLOPs)',
    date_start = '1950-01-01',
    date_end = '2022-02-01',

    # Era options
    start_dl_era = '2009-12-31',
    start_large_scale_era = '2015-09-01',
    split_dl_era = True,
    split_large_scale_era = True,

    # Data options
    citation_threshold = 0,
    separate_categories = False,
    other_domain_threshold = 10,
    outliers_action = 'remove',
    large_scale_action = 'label',
    big_alphago_action = 'label',
    record_setters_action = 'ignore',
    low_outliers_z_value_threshold = -2,
    high_outliers_z_value_threshold = 0.76,
    outlier_window_size = 2,

    # Bootstrapping options
    bootstrap_sample_size = 500,
    adjust_for_estimate_uncertainty = True,

    # Visualization options 
    label_points = False,
    plot_regressions = True,
    label_eras = False,
  ):

  # Preprocess parameters
  date_start = pd.to_datetime(date_start)
  date_end = pd.to_datetime(date_end)
  start_dl_era = np.datetime64(start_dl_era)
  start_large_scale_era = np.datetime64(start_large_scale_era)

  df = make_df(x_axis, y_axis, date_start, date_end, citation_threshold, separate_categories, other_domain_threshold)

  # Filter outliers
  df = filter_outliers(
    df, x_axis, y_axis,

    outliers_action, large_scale_action, 
    big_alphago_action, record_setters_action,

    low_outliers_z_value_threshold, high_outliers_z_value_threshold,
    outlier_window_size, start_large_scale_era
    )

  # Adjust date start and date end based on available data
  date_start = df['Publication date'].min()
  date_end = df['Publication date'].max()

  # Make dataframe with era information
  eras_df = make_eras_df(date_start, start_dl_era, start_large_scale_era, date_end)

  # Add eras info to dataset
  df = add_era_info(df, eras_df, split_dl_era, split_large_scale_era)

  # Regress on each domain
  df_reg, df_results =\
    regress_data(df, eras_df, x_axis, y_axis, bootstrap_sample_size, adjust_for_estimate_uncertainty)

  # Make charts
  chart =\
    make_chart(
      df, 
      eras_df,
      df_reg, 
      x_axis, 
      y_axis, 
      date_start,
      date_end,
      label_points, 
      plot_regressions, 
      label_eras,
      separate_categories,
      )

  # Return results
  return chart, df_results

def make_df(
    x_axis,
    y_axis,
    date_start, 
    date_end, 
    citation_threshold, 
    separate_categories,
    other_domain_threshold,
    ):
  # Download dataset
  df = pd.read_csv('https://docs.google.com/spreadsheets/d/1AAIebjNsnJj_uKALHbXNfn3_YsT6sHXtCU0q7OIPuc4/export?format=csv#gid=0')

  # Preprocess columns
  df['Publication date'] = pd.to_datetime(df['Publication date'], 
                                          errors='coerce', 
                                          dayfirst=True)

  df['Training compute per parameter (FLOPs)'] = \
    df['Training compute (FLOPs)'] / df['Parameters']

  df['Training compute times parameters'] = \
    df['Training compute (FLOPs)'] * df['Parameters']

  # Filter systems before and after date threshold
  df = df[df['Publication date'] > date_start]
  df = df[df['Publication date'] < date_end]

  # Filter systems below citation threshold
  df['Citations'] = df['Citations'].fillna(0)
  df = df[df['Citations'] >= citation_threshold]
  #df = df[(df['Citations'] >= citation_treshold) | df['Citations'].isna()]

  # Mask nans
  mask = df[x_axis].isna() | df[y_axis].isna() | (df[y_axis] == 0)
  df = df[~mask]

  # If not separating categories, delete all domain information
  if not separate_categories:
    df['Domain'] = "All"

  # Recode low count categories as "other"
  vc = df['Domain'].value_counts()
  sparse_domains = [domain for domain in vc.index if vc[domain]<other_domain_threshold]
  mask = df['Domain'].isin(sparse_domains)
  df.loc[mask,('Domain',)] = 'Other'

  return df

def filter_outliers(
    df, 
    x_axis,
    y_axis,

    outliers_action, 
    large_scale_action, 
    big_alphago_action,
    record_setters_action,

    low_outliers_z_value_threshold,
    high_outliers_z_value_threshold,
    outlier_window_size,
    start_large_scale_era,
    ):
  """
  Label, isolate or ignore outliers of various kinds
  """
  outliers_idx = set()
  large_scale_idx = set()

  for index, row in df.iterrows():
    # Filter entries in a 3-year window around the paper
    window_size = pd.Timedelta(f'{outlier_window_size*52*7} days')
    half_window_size = window_size / 2
    mask = ( row['Publication date'] - half_window_size <= df['Publication date'] ) &\
          ( df['Publication date'] <= row['Publication date'] + half_window_size )
    window_df = df[mask].copy()

    if len(window_df) < 2: continue

    for axis in (x_axis, y_axis):
      if axis == 'Publication date': continue
      window_df[f'{axis} z scores'] = stats.zscore(np.log10(window_df[axis].values))
      if window_df.loc[index, f'{axis} z scores'] < low_outliers_z_value_threshold: 
        outliers_idx.add(index)
      if window_df.loc[index, f'{axis} z scores'] > high_outliers_z_value_threshold: 
        large_scale_idx.add(index)

  # Drop low-scale outliers
  if outliers_action == 'remove':
    df = df.drop(outliers_idx)
  elif outliers_action == 'label':
    df.loc[outliers_idx, "Domain"] = "Outlier"

  # Drop large-scale outliers
  large_scale_mask = df.index.isin(large_scale_idx) & \
                    (df['Publication date'] > start_large_scale_era)
  if large_scale_action == 'label':
    df.loc[large_scale_mask, "Domain"] = "Large Scale"
  if large_scale_action == 'isolate':
    df = df[large_scale_mask]
    df["Domain"] = "Large Scale"

  # Drop alphago zero
  if big_alphago_action == 'remove':
    df = df[~(df['System'] == "AlphaGo Zero")]
    df = df[~(df['System'] == "AlphaGo Master")]
  elif big_alphago_action == 'label':
    df.loc[df['System'] == "AlphaGo Zero", "Domain"] = 'AlphaGo Zero'
    df.loc[df['System'] == "AlphaGo Master", "Domain"] = 'AlphaGo Zero'
    
  # Filter record setter models
  record_setters_mask =\
    df.apply(lambda row : row[y_axis] >= df[(df['Publication date'] <= row['Publication date']) & 
                                             (df['Domain'] == row['Domain'])
                                            ][y_axis].max(), axis = 1)
  if record_setters_action == 'isolate':
    df = df[record_setters_mask]
  elif record_setters_action == 'label':
    df.loc[record_setters_mask, "Domain"] = "Record"
  
  return df

def make_eras_df(date_start, start_dl_era, start_large_scale_era, date_end):
  eras = [
    ('Pre Deep Learning Era', date_start, start_dl_era),
    (    'Deep Learning Era', start_dl_era, start_large_scale_era),
    (      'Large Scale Era', start_large_scale_era, date_end),
  ]
  eras_df = pd.DataFrame(eras, columns = ['Era', 'start', 'stop'])

  ## Remove eras outside of the considered timespan
  mask =\
  (eras_df['stop']  > date_start) & \
  (eras_df['start'] < date_end)
  eras_df = eras_df[mask]

  ## Modify eras start-stop to fit the considered timespan
  eras_df.loc[eras_df['start']  < date_start, 'start'] = date_start
  eras_df.loc[eras_df['stop']   > date_end, 'stop']  = date_end

  return eras_df

def find_era(eras_df, date):
    """ Find the era that corresponds to a given date
    """
    mask = (eras_df['start'] <= date) & (date <= eras_df['stop'])
    era = eras_df[mask]['Era'].iloc[0]
    return era

def add_era_info(df, eras_df, split_dl_era, split_large_scale_era):
  # Add era labels to each domain
  if split_dl_era:
    df['Era'] = df['Publication date'].apply(lambda date: find_era(eras_df, date))
  else:
    df['Era'] = 'Machine Learning Era'

  ## Merge large scale and DL era
  if not split_large_scale_era:
    mask = df['Era'] == 'Large Scale Era'
    df.loc[mask, 'Era'] = 'Deep Learning Era'

  ## Make the era label categorical so it can be sorted
  df['Era'] = pd.Categorical(df['Era'], eras_df['Era'])

  return df

def julian_date_to_datetime(jd):
  # https://stackoverflow.com/questions/41154423/convert-from-to-julian-date-to-np-datetime64
  epoch = pd.to_datetime(0, unit='s').to_julian_date()
  dt = pd.to_datetime(jd - epoch, unit='D')
  return dt.strftime('%Y-%m-%d')
julian_date_to_datetime = np.vectorize(julian_date_to_datetime)

def regress_data(
    df, 
    eras_df, 
    x_axis,
    y_axis,
    bootstrap_sample_size,
    adjust_for_estimate_uncertainty
    ):
  """ Prepares a dataframe with interpolations of the data
      and a dataset of summary statistics of the interpolations
  """
  regression_data = []
  results = []
  for (era, domain), group in df.groupby(['Era', 'Domain'], sort=True):
    # Extract data matrices
    if x_axis == 'Publication date':
      X = group['Publication date'].apply(lambda x : x.to_julian_date())
    else:
      X = group[x_axis]
    X = X.values.reshape(-1, 1)
    Y = group[y_axis].values

    # Preprocessing
    if x_axis != 'Publication date':
      X = np.log10(X) 
    Y = np.log10(Y)

    assert not np.any(np.isnan(X)) and not np.any(np.isnan(Y))

    if len(X) <= 2: continue

    # Fit linear model
    model = linear_model.LinearRegression().fit(X, Y)
    
    # Bootstrap
    
    bootstrap_slopes = []
    bootstrap_start_values = []
    bootstrap_end_values = []
    if x_axis == 'Publication date': 
      bootstrap_doubling_times = []
    for bootstrap in range(bootstrap_sample_size):
      idx_bootstrap = np.random.choice(range(len(X)), size = len(X), replace=True)
      # We need at least 3 distinct points to do a linear regression
      if len(np.unique(idx_bootstrap)) <= 2: continue
      X_bootstrap = X[idx_bootstrap].reshape(-1, 1)
      Y_bootstrap = Y[idx_bootstrap]
      assert X_bootstrap.shape == X.shape
      assert Y_bootstrap.shape == Y.shape
      if adjust_for_estimate_uncertainty:
        noise = np.random.uniform(np.log10(1/2), np.log10(2), size=Y.shape)
        Y_bootstrap += noise
      bootstrap_model = linear_model.LinearRegression().fit(X_bootstrap, Y_bootstrap)

      ## Append slope
      bootstrap_slopes.append(bootstrap_model.coef_[0])

      ## Append doubling time
      if x_axis == 'Publication date':
        DAYS_PER_MONTH = 30
        bootstrap_doubling_times.append(np.log10(2) / bootstrap_model.coef_[0] / DAYS_PER_MONTH)
      
      ## Append start and end value
      y_start = 10**bootstrap_model.predict(X.min().reshape(1, -1))[0]
      y_end = 10**bootstrap_model.predict(X.max().reshape(1, -1))[0]
      bootstrap_start_values.append(y_start)
      bootstrap_end_values.append(y_end)

    # Collect information about the fit
    row = {}
    row['Trend'] = f"{domain} {era}"
    row['n'] = len(X)

    ## Start and end value
    low_q = 0.025
    median_q = 0.5
    high_q = 0.975
    
    y_start = 10**model.predict(X.min().reshape(1, -1))[0]
    y_start_low = np.quantile(bootstrap_start_values, q=low_q)
    y_start_median = np.quantile(bootstrap_start_values, q=median_q)
    y_start_high = np.quantile(bootstrap_start_values, q=high_q)

    y_end = 10**model.predict(X.max().reshape(1, -1))[0]
    y_end_low = np.quantile(bootstrap_end_values, q=low_q)
    y_end_median = np.quantile(bootstrap_end_values, q=median_q)
    y_end_high = np.quantile(bootstrap_end_values, q=high_q)

    # row['start'] = f"{y_start:1.0e} [{y_start_low:1.0e} ; {y_start_median:1.0e} ; {y_start_high:1.0e}]"                       
    # row['end'] =   f"{y_end  :1.0e} [{y_end_low  :1.0e} ; {y_end_median  :1.0e} ; {y_end_high  :1.0e}]"
    row['Scale (start / end)'] = f'{y_start:1.0e} / {y_end:1.0e}'

    ## Slope
    best_slope = model.coef_[0]
    low_slope = np.quantile(bootstrap_slopes, q=low_q)
    median_slope = np.quantile(bootstrap_slopes, q=median_q)
    high_slope = np.quantile(bootstrap_slopes, q=high_q)
    
    if x_axis == 'Publication date':
      DAYS_PER_YEAR = 365
      best_slope *= DAYS_PER_YEAR
      low_slope *= DAYS_PER_YEAR
      median_slope *= DAYS_PER_YEAR
      high_slope *= DAYS_PER_YEAR
      slope_summary = f"{best_slope:0.1f} OOMs/year [{low_slope:0.1f} ; {median_slope:0.1f} ; {high_slope:0.1f}]"  
    else:
      slope_summary = f"{best_slope:0.1e} [{low_slope:0.1e} ; {median_slope:0.1e} ; {high_slope:0.1e}]"
    
    row["Slope"] = slope_summary

    ## Doubling time
    if x_axis == 'Publication date': 
      best_doubling_time = np.log10(2) / model.coef_[0] / DAYS_PER_MONTH
      low_doubling_time = np.quantile(bootstrap_doubling_times, q=low_q)
      median_doubling_time = np.quantile(bootstrap_doubling_times, q=median_q)
      high_doubling_time = np.quantile(bootstrap_doubling_times, q=high_q)
      doubling_time_summary = f"{best_doubling_time:0.1f} months [{low_doubling_time:0.1f} ; {median_doubling_time:0.1f} ; {high_doubling_time:0.1f}]"
      row["Doubling time"] = doubling_time_summary 
    results.append(row)

    ## R2
    row["R2"] = f"{model.score(X,Y):0.2f}"

    # Extract predictions
    if x_axis == 'Publication date':
      # Stretch datapoints to cover the corresponding era
      eps = 10
      min_t = julian_date_to_datetime(X.min()+eps)
      min_t = pd.to_datetime(min_t)
      min_era = eras_df[(eras_df['start'] <= min_t) & (min_t <= eras_df['stop'])].iloc[0]
      max_t = julian_date_to_datetime(X.max()-eps)
      max_t = pd.to_datetime(max_t)
      max_era = eras_df[(eras_df['start'] <= max_t) & (max_t <= eras_df['stop'])].iloc[0]
      x_pred = np.linspace(min_era['start'].to_julian_date(), max_era['stop'].to_julian_date(),200)
    else:
      x_pred = np.linspace(min(X), max(X))
    y_pred = model.predict(x_pred.reshape(-1, 1))
    # Postprocessing
    if x_axis == 'Publication date': 
      x_pred = julian_date_to_datetime(x_pred)
    else:
      x_pred = 10.0**x_pred

    y_pred = 10.0**y_pred

    # Store in dataframe
    regression_data += [{x_axis : x,
                        y_axis : y,
                        'Domain' : domain} 
                        for x,y in zip(x_pred, y_pred)]

  df_results = pd.DataFrame(results)
  df_reg = pd.DataFrame(regression_data)

  return df_reg, df_results

# plot the dataset, referencing dataframe column names
def make_chart(
    df, 
    eras_df,
    df_reg, 
    x_axis, 
    y_axis, 
    date_start,
    date_end,
    label_points, 
    plot_regressions, 
    label_eras,
    separate_categories,
    log_axis=True,
    ):
  """ Makes an altair chart of the data
  """
  alt.themes.enable('fivethirtyeight')
  selection = alt.selection_multi(fields=['Domain'], bind='legend')

  domain_key =  [
      ('Vision', '#6d904f', 'cross'), 
      ('Language', '#b96db8', 'square'),   
      ('Games', '#30a2da', 'circle'),
      ('Drawing', '#8b8b8b', 'M0,.5L.6,.8L.5,.1L1,-.3L.3,-.4L0,-1L-.3,-.4L-1,-.3L-.5,.1L-.6,.8L0,.5Z'),
      ('Speech', '#ff9e27', 'triangle-down'),
      ('Other', '#e5ae38', 'diamond'), 
      ('Large Scale', '#fc4f30', 'triangle'),     
      ('All', '#30a2da', 'circle'),
      ('Outlier', '#56cc60', 'triangle-left'),
      ('AlphaGo Zero', '#ff9e27', 'M0,.5L.6,.8L.5,.1L1,-.3L.3,-.4L0,-1L-.3,-.4L-1,-.3L-.5,.1L-.6,.8L0,.5Z'),
      ('Record', '#ff9e27', 'M0,.5L.6,.8L.5,.1L1,-.3L.3,-.4L0,-1L-.3,-.4L-1,-.3L-.5,.1L-.6,.8L0,.5Z')
  ]
  domain_df = pd.DataFrame(domain_key, columns = ['Domain', 'Color', 'Shape'])

  # Filter domains that are present 
  mask = domain_df['Domain'].apply(lambda domain : domain in df['Domain'].unique())
  domain_df = domain_df[mask]

  color_scale = alt.Scale(
          domain=domain_df['Domain'].values,
          range=domain_df['Color'].values,
          )

  shape_scale = alt.Scale(
      domain= domain_df['Domain'].values,
      range=domain_df['Shape'].values,
  )

  make_domain_color = lambda legend : alt.Color(
      'Domain', 
      legend= legend,
      scale= color_scale
      )

  ## Chart with historical data
  chart = alt.Chart(df, width=1100, height=600,)\
  .properties(
      title={
        "text": [
                f"{y_axis} of milestone Machine Learning systems over time"\
                if x_axis == 'Publication date' else\
                f"{y_axis} vs {x_axis} of milestone Machine Learning systems"
                ], 
        "subtitle": [f"n = {len(df)}"]
      }
  )\
  .mark_point(size=120, filled=False)\
  .encode(
    x=alt.X(f'{x_axis}:{"T" if x_axis == "Publication date" else "Q"}',
            scale=alt.Scale(type='time' if x_axis == "Publication date" else 'linear' if not log_axis else 'log', 
                            domain=(df[x_axis].min(), df[x_axis].max())), 
            axis=alt.Axis(
                format="%Y" if x_axis == "Publication date" else ".1e",

                values = [pd.to_datetime(f'{year}-01-01') for year in range(date_start.year, date_end.year + 2)] \
                        if x_axis == "Publication date" else alt.Undefined
                )
            ),
    y=alt.Y(f'{y_axis}:Q',
            scale=alt.Scale(type='log' if log_axis else 'linear', 
                            domain=(df[y_axis].min(), df[y_axis].max())), 
            axis=alt.Axis(format="e", grid = True)
            ),
    color= make_domain_color(legend = None if not separate_categories else alt.Legend()),
    shape = alt.Shape('Domain', 
                      legend = None if not separate_categories else alt.Legend(),
                      scale = shape_scale,
                      ),
    tooltip=['System', 
            'Reference', 
            'Publication date', 
            alt.Tooltip('Parameters', format=".1e"), 
            alt.Tooltip('Training compute (FLOPs)', format=".1e"), 
            alt.Tooltip('Inference compute (FLOPs)', format=".1e"), 
            'Domain'
            ],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
  )

  ## Labels
  if label_points:

    # Define text constructor
    make_text = lambda df_slice, text_align: alt.Chart(df_slice).mark_text(
        align='left' if text_align == 'right' else 'right',
        baseline='middle',
        dx=7 if text_align == 'right' else -7
    ).encode(
        x=alt.X(f'{x_axis}:{"T" if x_axis == "Publication date" else "Q"}',),
        y=alt.Y(f'{y_axis}:Q',),
        text='System',
        color=make_domain_color(legend=None),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    )

    # Label left and right points appropriately
    midpoint = pd.to_datetime(f"{(date_start.year + date_end.year) // 2}-01-01")
    left_text = make_text(df[df['Publication date'] < midpoint],'right')
    right_text = make_text(df[df['Publication date'] > midpoint],'left')

    # Combine with main chart
    chart = (chart + left_text + right_text)

  ## Eras

  if x_axis == 'Publication date' and label_eras:
    # Define era colors
    era_color = alt.Color(
        'Era', 
        legend = None, 
        scale=alt.Scale(
            domain=['Pre Deep Learning Era', 'Deep Learning Era', 'Large Scale Era'],
            range=['#e5ae38', '#30a2da', '#fc4f30']
            )
        )
    
    # Create era backgrounds
    eras_chart = alt.Chart(
        eras_df
    ).mark_rect(
        opacity=0.1
    ).encode(
        x=alt.X('start', title = x_axis),
        x2=alt.X2('stop'),
        y=alt.value(0),  # pixels from top
        y2=alt.value(1000),  # pixels from top
        color=era_color
    )

    # Era label constructor
    make_era_labels = lambda df_slice, mode : \
      alt.Chart(df_slice).mark_text(
          align='right' if mode == 'high' else 'left',
          baseline='top',
          size = 18,
          fontWeight='bold',
          angle = 270,
          dx = -20,#120,
          dy=20,#25
      ).encode(
        text='Era',
        x=alt.X('start', title = x_axis),
        y=alt.value(0 if mode =="high" else 570),  # pixels from top
        color=era_color
      )

    # Instantiate left and right era labels
    midpoint = pd.to_datetime(f"{(date_start.year + date_end.year) // 2}-01-01")
    left_era_labels = make_era_labels(eras_df[eras_df['start'] < midpoint], "high")
    right_era_labels = make_era_labels(eras_df[eras_df['start'] > midpoint], "low")

    # Combine with main chart
    chart = (chart + eras_chart + left_era_labels + right_era_labels)

  ## Chart with regressions
  if plot_regressions:
    reg_chart = alt.Chart(df_reg).mark_line(point=False, strokeDash=[10,5], clip=True)\
    .encode(
        x=f'{x_axis}:{"T" if x_axis == "Publication date" else "Q"}',
        y=y_axis,
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
        color=make_domain_color(legend=None),
        )
    chart = (chart + reg_chart)

  # Final touches
  chart = chart\
  .add_selection(selection)\
  .configure_title(fontSize=42)\
  .configure(background='#FFFFFF')\
  .configure_axis(
      labelFontSize=20,titleFontSize=26)\
  .configure_legend(
      titleFontSize=20,
      labelFontSize =18,
      gradientLength=400,
      gradientThickness=30,
      symbolSize = 130,)\
  .resolve_scale(shape='independent', color='independent')\
  .interactive()

  return chart

##############################################################################

chart, df_results = \
  make_visualization(
      x_axis,
      y_axis,
      date_start,
      date_end,

      # Era options
      start_dl_era,
      start_large_scale_era,
      split_dl_era,
      split_large_scale_era,

      # Data options
      citation_threshold,
      separate_categories,
      other_domain_threshold,
      outliers_action,
      large_scale_action,
      big_alphago_action,
      record_setters_action,
      low_outliers_z_value_threshold,
      high_outliers_z_value_threshold,
      outlier_window_size,

      # Bootstrapping options
      bootstrap_sample_size,
      adjust_for_estimate_uncertainty,

      # Visualization options 
      label_points,
      plot_regressions,
      label_eras,
  )

display(df_results)
chart

,Trend,n,Scale (start / end),Slope,Doubling time,R2
0,All Pre Deep Learning Era,9,4e+03 / 1e+14,0.2 OOMs/year [0.2 ; 0.2 ; 0.3],17.5 months [12.2 ; 17.3 ; 21.9],0.90
1,Record Pre Deep Learning Era,10,1e+05 / 3e+14,0.2 OOMs/year [0.1 ; 0.2 ; 0.2],19.9 months [14.7 ; 20.3 ; 30.9],0.83
2,All Deep Learning Era,5,3e+14 / 7e+15,0.6 OOMs/year [0.1 ; 0.6 ; 2.5],6.2 months [1.3 ; 6.0 ; 23.6],0.70
3,All Large Scale Era,85,1e+16 / 4e+22,0.7 OOMs/year [0.6 ; 0.7 ; 0.9],5.1 months [4.3 ; 5.1 ; 6.2],0.53
4,Record Large Scale Era,12,6e+18 / 9e+24,0.8 OOMs/year [0.6 ; 0.8 ; 1.6],4.7 months [2.3 ; 4.6 ; 6.3],0.78


alt.LayerChart(...)

In [ ]:
## Run this first!
!pip install altair_saver --quiet
!npm install vega-lite vega-cli canvas --silent

     |████████████████████████████████| 89 kB 4.8 MB/s 
     |████████████████████████████████| 958 kB 15.7 MB/s 
     |████████████████████████████████| 844 kB 52.8 MB/s 
     |████████████████████████████████| 356 kB 15.1 MB/s 
     |████████████████████████████████| 138 kB 45.9 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 3.6 MB 47.7 MB/s 
     |████████████████████████████████| 58 kB 4.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
[canvas] Success: "/content/node_modules/canvas/build/Release/canvas.node" is installed via remote
+ canvas@2.9.0
+ vega-lite@5.2.0
+ vega-cli@5.21.0
added 14

In [ ]:
from numpy.core.numeric import False_
#@title Automatically generate charts and results

settings_list = [
  # {
  #     'chart_name' : 'fig1.png',
  #     'table_name' : 'fig1table.csv',
  #     'visualization_params' : {
  #         'x_axis' : 'Publication date',
  #         'y_axis' : 'Training compute (FLOPs)',
  #         'date_start' : '1950-01-01',
  #         'date_end' : '2022-02-01',

  #         # Era options
  #         'start_dl_era' : '2009-12-31',
  #         'start_large_scale_era' : '2015-09-01',
  #         'split_dl_era' : True,
  #         'split_large_scale_era' : True,

  #         # Data options
  #         'citation_threshold' : 0,
  #         'separate_categories' : False,
  #         'other_domain_threshold' : 10,
  #         'outliers_action' : 'remove',
  #         'large_scale_action' : 'label',
  #         'big_alphago_action' : 'ignore',
  #         'record_setters_action' : 'ignore',
  #         'low_outliers_z_value_threshold' : -2,
  #         'high_outliers_z_value_threshold' : 0.76,
  #         'outlier_window_size' : 2,

  #         # Bootstrapping options
  #         'bootstrap_sample_size' : 1000,
  #         'adjust_for_estimate_uncertainty' : True,

  #         # Visualization options 
  #         'label_points' : False,
  #         'plot_regressions' : True,
  #         'label_eras' : True,
  #     }
  # },
  # {
  #     'chart_name' : 'fig2.png',
  #     'table_name' : 'fig2table.csv',
  #     'visualization_params' : {
  #         'x_axis' : 'Publication date',
  #         'y_axis' : 'Training compute (FLOPs)',
  #         'date_start' : '1950-01-01',
  #         'date_end' : '2022-02-01',

  #         # Era options
  #         'start_dl_era' : '2009-12-31',
  #         'start_large_scale_era' : '2015-09-01',
  #         'split_dl_era' : True,
  #         'split_large_scale_era' : True,

  #         # Data options
  #         'citation_threshold' : 0,
  #         'separate_categories' : True,
  #         'other_domain_threshold' : 10,
  #         'outliers_action' : 'remove',
  #         'large_scale_action' : 'label',
  #         'big_alphago_action' : 'ignore',
  #         'record_setters_action' : 'ignore',
  #         'low_outliers_z_value_threshold' : -2,
  #         'high_outliers_z_value_threshold' : 0.76,
  #         'outlier_window_size' : 2,

  #         # Bootstrapping options
  #         'bootstrap_sample_size' : 1000,
  #         'adjust_for_estimate_uncertainty' : True,

  #         # Visualization options 
  #         'label_points' : False,
  #         'plot_regressions' : False,
  #         'label_eras' : True,
  #     }
  # },
  # {
  #     'chart_name' : 'fig3.png',
  #     'table_name' : 'fig3table.csv',
  #     'visualization_params' : {
  #         'x_axis' : 'Publication date',
  #         'y_axis' : 'Training compute (FLOPs)',
  #         'date_start' : '1950-01-01',
  #         'date_end' : '2022-02-01',

  #         # Era options
  #         'start_dl_era' : '2009-12-31',
  #         'start_large_scale_era' : '2024-09-01',
  #         'split_dl_era' : True,
  #         'split_large_scale_era' : True,

  #         # Data options
  #         'citation_threshold' : 0,
  #         'separate_categories' : False,
  #         'other_domain_threshold' : 10,
  #         'outliers_action' : 'remove',
  #         'large_scale_action' : 'ignore',
  #         'big_alphago_action' : 'ignore',
  #         'record_setters_action' : 'ignore',
  #         'low_outliers_z_value_threshold' : -2,
  #         'high_outliers_z_value_threshold' : 0.76,
  #         'outlier_window_size' : 2,

  #         # Bootstrapping options
  #         'bootstrap_sample_size' : 1000,
  #         'adjust_for_estimate_uncertainty' : True,

  #         # Visualization options 
  #         'label_points' : False,
  #         'plot_regressions' : True,
  #         'label_eras' : True,
  #     }
  # },
  # {
  #   'chart_name' : 'fig4.png',
  #   'table_name' : 'fig4table.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2009-12-31',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-31',
  #       'start_large_scale_era' : '2015-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : True,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'fig5.png',
  #   'table_name' : 'fig5table.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2015-09-01',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-31',
  #       'start_large_scale_era' : '2015-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'isolate',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : True,
  #       'plot_regressions' : True,
  #       'label_eras' : False,
  #   }
  # },
  # {
  #   'chart_name' : 'fig6.png',
  #   'table_name' : 'fig6table.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '1990-01-01',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-31',
  #       'start_large_scale_era' : '2015-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'label',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : False,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'fig7.png',
  #   'table_name' : 'fig7table.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '1950-01-01',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-31',
  #       'start_large_scale_era' : '2015-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'ignore',
  #       'large_scale_action' : 'ignore',
  #       'big_alphago_action' : 'remove',
  #       'record_setters_action' : 'isolate',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : True,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'fig8.png',
  #   'table_name' : 'fig8table.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2009-12-31',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-31',
  #       'start_large_scale_era' : '2015-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : False,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : True,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'ignore',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  #   {
  #   'chart_name' : 'fig10.png',
  #   'table_name' : 'fig10table.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2012-09-01',
  #       'date_end' : '2017-12-01',

  #       # Era options
  #       'start_dl_era' : '2012-09-01',
  #       'start_large_scale_era' : '2015-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : False,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table1run1.png',
  #   'table_name' : 'table1run1.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '1950-01-01',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-31',
  #       'start_large_scale_era' : '2015-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'ignore',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table1run2.png',
  #   'table_name' : 'table1run2.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '1950-01-01',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-31',
  #       'start_large_scale_era' : '2009-12-31',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table1run3.png',
  #   'table_name' : 'table1run3.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '1950-01-01',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-31',
  #       'start_large_scale_era' : '2015-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table4run1.png',
  #   'table_name' : 'table4run1.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '1950-01-01',
  #       'date_end' : '2022-01-01',

  #       # Era options
  #       'start_dl_era' : '2009-09-01',
  #       'start_large_scale_era' : '2022-03-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table4run2.png',
  #   'table_name' : 'table4run2.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '1950-01-01',
  #       'date_end' : '2022-01-01',

  #       # Era options
  #       'start_dl_era' : '2009-09-01',
  #       'start_large_scale_era' : '2009-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table4run3.png',
  #   'table_name' : 'table4run3.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '1950-01-01',
  #       'date_end' : '2022-01-01',

  #       # Era options
  #       'start_dl_era' : '2012-09-30',
  #       'start_large_scale_era' : '2022-03-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table4run4.png',
  #   'table_name' : 'table4run4.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '1950-01-01',
  #       'date_end' : '2022-01-01',

  #       # Era options
  #       'start_dl_era' : '2012-09-30',
  #       'start_large_scale_era' : '2012-09-30',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table5run1.png',
  #   'table_name' : 'table5run1.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2009-12-31',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-31',
  #       'start_large_scale_era' : '2015-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table5run2.png',
  #   'table_name' : 'table5run2.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2015-09-01',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-31',
  #       'start_large_scale_era' : '2024-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table6run1.png',
  #   'table_name' : 'table6run1.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '1950-01-01',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-31',
  #       'start_large_scale_era' : '2015-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'ignore',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'label',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table6run2.png',
  #   'table_name' : 'table6run2.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '1950-01-01',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-31',
  #       'start_large_scale_era' : '2015-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'ignore',
  #       'big_alphago_action' : 'remove',
  #       'record_setters_action' : 'label',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table7run1.png',
  #   'table_name' : 'table7run1.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2009-12-31',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-30',
  #       'start_large_scale_era' : '2015-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : False,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : True,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'ignore',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table7run2.png', 
  #   'table_name' : 'table7run2.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2009-12-31',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-30',
  #       'start_large_scale_era' : '2009-12-30',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : True,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table7run2.png', # struggling to replicate
  #   'table_name' : 'table7run2.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2009-12-31',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2009-12-30',
  #       'start_large_scale_era' : '2009-12-30',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : True,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table8run1.png', 
  #   'table_name' : 'table8run1.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2012-09-01',
  #       'date_end' : '2017-12-01',

  #       # Era options
  #       'start_dl_era' : '2012-09-01',
  #       'start_large_scale_era' : '2012-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'ignore',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table8run2.png', 
  #   'table_name' : 'table8run2.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2012-09-01',
  #       'date_end' : '2017-12-01',

  #       # Era options
  #       'start_dl_era' : '2012-09-01',
  #       'start_large_scale_era' : '2012-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table8run3.png', 
  #   'table_name' : 'table8run3.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2017-12-01',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2017-11-30',
  #       'start_large_scale_era' : '2017-11-30',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'ignore',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table8run4.png', 
  #   'table_name' : 'table8run4.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2017-12-01',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2017-11-30',
  #       'start_large_scale_era' : '2017-11-30',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table8run5.png', 
  #   'table_name' : 'table8run5.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2012-09-01',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2012-09-01',
  #       'start_large_scale_era' : '2012-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'ignore',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },
  # {
  #   'chart_name' : 'table8run6.png', 
  #   'table_name' : 'table8run6.csv',
  #   'visualization_params' : {
  #       'x_axis' : 'Publication date',
  #       'y_axis' : 'Training compute (FLOPs)',
  #       'date_start' : '2012-09-01',
  #       'date_end' : '2022-02-01',

  #       # Era options
  #       'start_dl_era' : '2012-09-01',
  #       'start_large_scale_era' : '2012-09-01',
  #       'split_dl_era' : True,
  #       'split_large_scale_era' : True,

  #       # Data options
  #       'citation_threshold' : 0,
  #       'separate_categories' : False,
  #       'other_domain_threshold' : 10,
  #       'outliers_action' : 'remove',
  #       'large_scale_action' : 'label',
  #       'big_alphago_action' : 'ignore',
  #       'record_setters_action' : 'ignore',
  #       'low_outliers_z_value_threshold' : -2,
  #       'high_outliers_z_value_threshold' : 0.76,
  #       'outlier_window_size' : 2,

  #       # Bootstrapping options
  #       'bootstrap_sample_size' : 1000,
  #       'adjust_for_estimate_uncertainty' : True,

  #       # Visualization options 
  #       'label_points' : False,
  #       'plot_regressions' : True,
  #       'label_eras' : True,
  #   }
  # },  
  {
    'chart_name' : 'table8run7.png', 
    'table_name' : 'table8run7.csv',
    'visualization_params' : {
        'x_axis' : 'Publication date',
        'y_axis' : 'Training compute (FLOPs)',
        'date_start' : '2015-09-01',
        'date_end' : '2022-02-01',

        # Era options
        'start_dl_era' : '2012-09-01',
        'start_large_scale_era' : '2015-09-01',
        'split_dl_era' : True,
        'split_large_scale_era' : True,

        # Data options
        'citation_threshold' : 0,
        'separate_categories' : False,
        'other_domain_threshold' : 10,
        'outliers_action' : 'remove',
        'large_scale_action' : 'label',
        'big_alphago_action' : 'ignore',
        'record_setters_action' : 'ignore',
        'low_outliers_z_value_threshold' : -2,
        'high_outliers_z_value_threshold' : 0.76,
        'outlier_window_size' : 2,

        # Bootstrapping options
        'bootstrap_sample_size' : 1000,
        'adjust_for_estimate_uncertainty' : True,

        # Visualization options 
        'label_points' : False,
        'plot_regressions' : True,
        'label_eras' : True,
    }
  },  
]

#@markdown Enter number from 0 to N-1, where N is the number of settings
#@markdown Enter 'all' to show all charts and tables
display_index = 'all' #@param

!rm output -r
!mkdir output

if display_index == 'all':

  
  for settings in settings_list:
    chart, df_results = make_visualization(**settings['visualization_params'])
    chart.save('output/' + settings['chart_name'])
    df_results.to_csv('output/' + settings['table_name'])
    display(chart)
    display(df_results)
  
else:
  settings = settings_list[display_index]
  chart, df_results = make_visualization(**settings['visualization_params'])
  chart.save('output/' + settings['chart_name'])
  df_results.to_csv('output/' + settings['table_name'])
  display(chart)
  display(df_results)



rm: cannot remove 'output': No such file or directory


/usr/local/lib/python3.7/dist-packages/altair/utils/core.py:221: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  "Defaulting to nominal.".format(typ)


alt.LayerChart(...)

,Trend,n,Scale (start / end),Slope,Doubling time,R2
0,All Large Scale Era,60,3e+18 / 2e+22,0.6 OOMs/year [0.5 ; 0.6 ; 0.8],5.8 months [4.5 ; 5.8 ; 8.0],0.48
1,Large Scale Large Scale Era,19,4e+21 / 6e+23,0.3 OOMs/year [0.1 ; 0.3 ; 0.5],10.7 months [7.8 ; 10.7 ; 27.2],0.66


In [ ]:
# Scratchpad
t_step = pd.DateOffset(months=1)
window_size = pd.DateOffset(months=18)
vars = []
for i_date in pd.date_range(date_start, date_end, freq=t_step):
  # Select data from the period 
  mask = (i_date - window_size <= df['Publication date']) & (df['Publication date'] < i_date + t_step)
  step_df = df[mask]
  var = step_df[y_axis].apply(lambda x: np.log10(x)).var()
  vars.append({
      'Date' : i_date,
      'Rolling variance': var,
      })

vars_df = pd.DataFrame(vars)

alt.Chart(vars_df).mark_line().encode(x='Date', y='Rolling variance')

alt.Chart(...)